# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import sqlite3 as sq3
import pandas as pd

In [2]:
# Your code here; create the connection
conn = sq3.connect('data.sqlite')

def exqr(query):
    return pd.read_sql(query, conn)

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [3]:
# Your code here
exqr('''
    SELECT
        customerNumber,
        contactLastName,
        contactFirstName
    FROM customers
    WHERE customerNumber IN(SELECT
                                customerNumber
                            FROM 
                                orders
                            WHERE orderDate = '2003-01-31'
                            );
''')

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
# how to get product name if not joined?
#select productName from products, count(order), where productCode in orderDetails, is 

exqr('''
      SELECT productName, productCode, SUM(quantityOrdered) AS TotalCount
      FROM orderDetails
      JOIN products
      USING(productCode)
 
     
      
      GROUP BY productName
      ORDER BY TotalCount;                                  
      
''')


,productName,productCode,TotalCount
0,1957 Ford Thunderbird,S18_4933,767
1,1970 Chevy Chevelle SS 454,S24_1046,803
2,1936 Mercedes Benz 500k Roadster,S24_3969,824
3,1911 Ford Town Car,S18_2248,832
4,1999 Indy 500 Monte Carlo SS,S18_2870,855
...,...,...,...
104,1930 Buick Marquette Phaeton,S50_1341,1074
105,1941 Chevrolet Special Deluxe Cabriolet,S18_3856,1076
106,American Airlines: MD-11S,S700_4002,1085
107,1937 Lincoln Berline,S18_1342,1111


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [9]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
# productName is on the products table. number of people is COUNT(customerNumber) FROM the orders table,
#WHERE orderNumber on orderDetails  has productCode
exqr("""
    SELECT productName, COUNT(orderNumber) AS people_who_ordered
    FROM products
    JOIN orderDetails
    USING(productCode)
    WHERE orderNumber IN(
        SELECT DISTINCT orderNumber
        FROM orders
        WHERE customerNumber IS NOT null)
    
    GROUP BY productName
    ORDER BY people_who_ordered
    DESC
""")

,productName,people_who_ordered
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
104,1932 Alfa Romeo 8C2300 Spider Sport,25
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24


<img src="images/Database-Schema.png" width="600">

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [16]:
# Your code here
exqr('''
    SELECT employeeNumber, firstName, lastName, offices.city, offices.officeCode, COUNT(customers.customerNumber) as who_ordered
    FROM employees
    JOIN offices
    USING(officeCode)
    JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
    WHERE customerNumber IN(
        SELECT customerNumber
        FROM orders
        GROUP BY 1
        HAVING COUNT(orderNumber) < 20 )
    GROUP BY employeeNumber;
''')

,employeeNumber,firstName,lastName,city,officeCode,who_ordered
0,1165,Leslie,Jennings,San Francisco,1,6
1,1166,Leslie,Thompson,San Francisco,1,6
2,1188,Julie,Firrelli,Boston,2,6
3,1216,Steve,Patterson,Boston,2,6
4,1286,Foon Yue,Tseng,NYC,3,6
5,1323,George,Vanauf,NYC,3,8
6,1337,Loui,Bondur,Paris,4,6
7,1370,Gerard,Hernandez,Paris,4,6
8,1401,Pamela,Castillo,Paris,4,10
9,1501,Larry,Bott,London,7,8


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [ ]:
# Your code here
exqr('''
    SELECT employeeNumber, firstName, lastName, offices.city, offices.officeCode, COUNT(customers.customerNumber) as who_ordered
    FROM employees
    JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
    WHERE customerNumber IN(
        SELECT customerNumber
        FROM orders
        GROUP BY 1
        HAVING COUNT(orderNumber) < 20 )
    GROUP BY employeeNumber;
''')

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!